## Corte de Usuários Mailchimp

## 1. Importando Pacotes

In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob, os
import time

## 1. Abrindo Bancos

campaings: vem do export da conta do mailchimp. Opens e N_opens vem da seleção manual de todos os arquivos a partir de julho de 2019 das pastas de aggregate_activity, que também vem do export da conta. Por último, inscritos vem do export da audiência no mailchimp, apenas pra selecionarmos pra corte quem tá lá ainda e evitar quem é de outras comunidades.

In [52]:
files = glob.glob('/home/jovyan/work/mailchimp_dump/granular_activity/opens/*.csv')
campaigns = pd.concat([pd.read_csv(fp, encoding='latin-1', nrows=1).assign(Campaign=os.path.basename(fp) ) 
                for fp in files], sort=False)
campaigns = campaigns[['Timestamp', 'Campaign']]

##### utilizando dados dos usários a partir de julho de 2019
campaigns[campaigns['Timestamp'] > '2019-07-01 00:00:01'].sort_values('Timestamp').head(1)

### 1.1 Aberturas

In [54]:
start_time = time.time()

files_opened = glob.glob('/home/jovyan/work/mailchimp_refined/recent_opened/*.csv')
opens = pd.concat([pd.read_csv(fp, encoding='latin-1',usecols = [0,1,2]
           ).assign(Campaign=os.path.basename(fp) ) for fp in files_opened], sort=False)

print("--- %s seconds ---" % (time.time() - start_time))

--- 9.940843105316162 seconds ---


In [55]:
open_1 = opens[['Email', 'Nome', 'Sobrenome', 'Campaign']].dropna(subset=['Email'])
open_2 = opens[['Email Address', 'First Name', 'Last Name', 'Campaign']].dropna(subset=['Email Address'])
open_2.columns = ['Email', 'Nome', 'Sobrenome', 'Campaign']
opens = pd.concat([open_1, open_2])

opens = opens.merge(campaigns,on='Campaign')
opens['Atividade'] = 'abertura'

### 1.2 Não aberturas

In [56]:
start_time = time.time()

files_notopened = glob.glob('/home/jovyan/work/mailchimp_refined/recent_not_opened/*.csv')
n_opens = pd.concat([pd.read_csv(fp, encoding='latin-1',usecols = [0,1,2]
           ).assign(Campaign=os.path.basename(fp) ) for fp in files_notopened], sort=False)

print("--- %s seconds ---" % (time.time() - start_time))

--- 28.214014768600464 seconds ---


In [57]:
n_open_1 = n_opens[['Email', 'Nome', 'Sobrenome', 'Campaign']].dropna(subset=['Email'])
n_open_2 = n_opens[['Email Address', 'First Name', 'Last Name', 'Campaign']].dropna(subset=['Email Address'])
n_open_2.columns = ['Email', 'Nome', 'Sobrenome', 'Campaign']
n_opens = pd.concat([n_open_1, n_open_2])


n_opens = n_opens.merge(campaigns,on='Campaign')
n_opens['Atividade'] = 'não abertura'

### 2. Unindo dados

In [58]:
all_activities = pd.concat([opens, n_opens])
all_activities = all_activities.sort_values(['Email','Timestamp'])
all_activities.to_csv('all_recent_activities.csv', index=False)

### 2.1 Mantendo Apenas Inscritos
Selecionando apenas inscritos + identificando quais inscrições

In [67]:
#df = pd.read_csv('all_recent_activities.csv')
df = all_activities.copy()

In [70]:
inscritos = pd.read_csv("/home/jovyan/work/members/subscribed.csv", usecols = ['Email', 'Inscrições', 'MEMBER_RATING'])
df = df.merge(inscritos, on='Email')
df['Inscrições'] = df['Inscrições'].fillna('0')

In [94]:
df['corte'] = np.where((df['Inscrições'].str.contains('Meu Recife') |df['Inscrições'].str.contains('Minha Jampa') 
         | df['Inscrições'].str.contains('Minha Campinas') | df['Inscrições'].str.contains('Minha Porto Alegre'))
         , "imune", "elegível")
df = df[df['corte'] == 'elegível']

### 3. Criando banco no nível do usuário

In [164]:
user_nopen = df[df['Atividade'] == 'não abertura'].groupby('Email', as_index=False).agg({'Atividade': "count"})
user_open = df[df['Atividade'] == 'abertura'].groupby('Email', as_index=False).agg({'Atividade': "count"})
user_geral = df.groupby('Email', as_index=False).agg({"Timestamp":"first","Campaign" : "first" ,"Inscrições":"last","MEMBER_RATING":"last"})
user_nopen.columns = ['Email','n_open']
user_open.columns = ['Email', 'open']
user_geral.columns = ['Email', 'First Email', 'Campaign', 'Inscrições', 'Member Rating']
user = pd.merge(user_nopen, user_open, on='Email', how='outer', indicator=True)
user = user.merge(user_geral, on='Email')

In [165]:
user['n_open'] = user['n_open'].fillna(0)
user['open'] = user['open'].fillna(0)
user['open_rate'] = user['open']/(user['open'] + user['n_open'])
user['total_emails'] = user['open'] + user['n_open']

### 4. Cortes

#### 4.1 Primeiro corte: Quem nunca abriu um email

In [166]:
user[(user['open_rate'] == 0)].shape

(428273, 10)

In [168]:
inativos = user[(user['open_rate'] == 0) & (user['total_emails'] >= 3)][['Email']] ## abriram nenhum e já receberam ao menos 3
inativos_velhos = user[(user['open_rate'] == 0) & (user['First Email'] < '2020-01-01 13:35:47')][[
    'Email']] # abriram 0 e são mto antigos
inativos = pd.concat([inativos, inativos_velhos])
inativos.Email.nunique() # 250006 salvos pq receberam menos de 3 emails.

178267

#### 4.2 Segundo corte: quem já abriu pelo menos um email

In [135]:
user = user[(user['open_rate'] != 0)]

In [180]:
df_3 = df.groupby('Email').tail(3) #last 3 rows
df_4 = df.groupby('Email').tail(4) #last 3 rows
df_5 = df.groupby('Email').tail(5) #last 5 rows
df_10 = df.groupby('Email').tail(10) #last 10 rows

In [181]:
df_3['abertura'] = np.where((df_3['Atividade'] =='abertura') ,1, 0)
df_3['não abertura'] = np.where((df_3['Atividade'] == 'não abertura'), 1, 0)

df_4['abertura'] = np.where((df_4['Atividade'] =='abertura') ,1, 0)
df_4['não abertura'] = np.where((df_4['Atividade'] == 'não abertura'), 1, 0)

df_5['abertura'] = np.where((df_5['Atividade'] =='abertura') ,1, 0)
df_5['não abertura'] = np.where((df_5['Atividade'] == 'não abertura'), 1, 0)

df_10['abertura'] = np.where((df_10['Atividade'] =='abertura') ,1, 0)
df_10['não abertura'] = np.where((df_10['Atividade'] == 'não abertura'), 1, 0)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [182]:
df_3 = df_3.groupby(['Email', 'Inscrições'],as_index=False).agg({'Atividade': "count", 'abertura': 'sum', 'não abertura':'sum'})
df_4 = df_4.groupby(['Email', 'Inscrições'],as_index=False).agg({'Atividade': "count", 'abertura': 'sum', 'não abertura':'sum'})
df_5 = df_5.groupby(['Email', 'Inscrições'],as_index=False).agg({'Atividade': "count", 'abertura': 'sum', 'não abertura':'sum'})
df_10=df_10.groupby(['Email', 'Inscrições'],as_index=False).agg({'Atividade': "count", 'abertura': 'sum', 'não abertura':'sum'})

In [185]:
apagar_3 = df_3[(df_3['Atividade'] == 3) & (df_3['não abertura'] == 3)]
apagar_4 = df_5[(df_4['Atividade'] == 4) & (df_4['não abertura'] == 4)]
apagar_5 = df_5[(df_5['Atividade'] == 5) & (df_5['não abertura'] == 5)]
apagar_10 = df_10[(df_10['Atividade'] == 10) & (df_10['não abertura'] == 10)]

### 5. Decidindo quem vai ser cortado

In [229]:
print(pd.concat([inativos, apagar_5]).Email.nunique())
print('+',pd.concat([inativos, apagar_4]).Email.nunique()  - 404280)
print('+',pd.concat([inativos, apagar_3]).Email.nunique() - 404280)
print('+',pd.concat([inativos, apagar_5, user[(user['open_rate'] == 0) & (user['total_emails'] == 1)]]).Email.nunique() - 404280)
print('+',pd.concat([inativos, apagar_5, user[(user['open_rate'] == 0) & (user['total_emails'] == 2)]]).Email.nunique() - 404280)  

404280
+ 56151
+ 145634
+ 171663
+ 78343


Diretoria solicitou remover os 145 a mais que não abriram os 3 últimos emails, mas por segurança, mantive quem tem mais de 30% de abertura entre esses.

In [263]:
a3_opens = apagar_3.merge(user[['Email', 'open_rate']], how='left')
a3 = a3[a3['open_rate']<0.3]

Corte opção 1: todo mundo que não abriu nenhum email e recebeu mas de 3 ou é mais antigo que 8 meses na base. Entre quem já abriu um email, sai quem já recebeu pelo menos 3 emails e não abriu nenhum dos últimos 3, exceto os que tem open rate geral maior que 30%. 

<br>

Corte opção 2: todo mundo que não abriu nenhum email e recebeu mas de 3 ou é mais antigo que 8 meses na base. Entre quem já abriu um email, sai quem já recebeu pelo menos 3 emails e não abriu nenhum dos últimos 3.

In [276]:
corte_opcao1 = pd.concat([inativos, apagar_5, a3.iloc[:,:-1]])[['Email']].drop_duplicates()
corte_opcao2 = pd.concat([inativos, apagar_5, apagar_3])[['Email']].drop_duplicates()

In [279]:
corte_opcao1.to_csv('corte_leve.csv', index=False)
corte_opcao2.to_csv('corte_severo.csv', index=False)

In [ ]:
corte_opcao1.shape